<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
## Task 1: Mark all launch sites# Importación de bibliotecas necesarias
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Datos de los sitios de lanzamiento
launch_sites = [
    {"name": "Cape Canaveral LC-40", "lat": 28.5623, "lon": -80.5774},
    {"name": "Vandenberg SLC-4E", "lat": 34.6321, "lon": -120.6106},
    {"name": "Cape Canaveral LC-39A", "lat": 28.5734, "lon": -80.6469},
    {"name": "Kourou ELA-3", "lat": 5.236, "lon": -52.768},
]

# Crear un mapa centrado en una ubicación específica
mapa = folium.Map(location=[20, -50], zoom_start=3)

# Agregar el plugin MarkerCluster
marker_cluster = MarkerCluster().add_to(mapa)

# Agregar marcadores de sitios de lanzamiento al cluster
for site in launch_sites:
    folium.Marker(
        location=[site["lat"], site["lon"]],
        popup=folium.Popup(f"Launch Site: {site['name']}", max_width=200),
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(marker_cluster)

# Agregar MousePosition para mostrar latitud/longitud
MousePosition(
    position="topright",
    separator=" | ",
    prefix="Coordenadas:",
    num_digits=5,
    empty_string="Sin posición"
).add_to(mapa)

# Agregar etiquetas usando DivIcon
for site in launch_sites:
    folium.Marker(
        location=[site["lat"], site["lon"]],
        icon=DivIcon(
            icon_size=(150, 30),  # Tamaño de la etiqueta
            icon_anchor=(0, 0),  # Posición de anclaje
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{site["name"]}</div>'
        )
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML (opcional)
mapa.save("launch_sites_map.html")

# Mostrar el mapa en el entorno interactivo
mapa
 on a map


<class 'IndentationError'>: unexpected indent (<ipython-input-6-06dd7d186e1c>, line 53)

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [9]:
# Task 2: Mark the success/failed launches for each site on the map
# Importación de bibliotecas necesarias
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Datos de los sitios de lanzamiento y detalles de los lanzamientos
launch_sites = [
    {
        "name": "Cape Canaveral LC-40",
        "lat": 28.5623,
        "lon": -80.5774,
        "launches": [
            {"date": "2023-01-01", "status": "success"},
            {"date": "2023-02-15", "status": "failure"},
        ],
    },
    {
        "name": "Vandenberg SLC-4E",
        "lat": 34.6321,
        "lon": -120.6106,
        "launches": [
            {"date": "2023-03-05", "status": "success"},
            {"date": "2023-04-10", "status": "success"},
        ],
    },
    {
        "name": "Cape Canaveral LC-39A",
        "lat": 28.5734,
        "lon": -80.6469,
        "launches": [
            {"date": "2023-05-20", "status": "failure"},
            {"date": "2023-06-01", "status": "success"},
        ],
    },
    {
        "name": "Kourou ELA-3",
        "lat": 5.236,
        "lon": -52.768,
        "launches": [
            {"date": "2023-07-12", "status": "success"},
            {"date": "2023-08-05", "status": "failure"},
        ],
    },
]

# Crear un mapa centrado en una ubicación específica
mapa = folium.Map(location=[20, -50], zoom_start=3)

# Agregar el plugin MarkerCluster
marker_cluster = MarkerCluster().add_to(mapa)

# Función para seleccionar color según el estado
def get_icon_color(status):
    return "green" if status == "success" else "red"

# Agregar lanzamientos al mapa
for site in launch_sites:
    # Etiqueta del sitio
    folium.Marker(
        location=[site["lat"], site["lon"]],
        icon=DivIcon(
            icon_size=(150, 30),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{site["name"]}</div>'
        )
    ).add_to(mapa)
    
    # Agregar lanzamientos individuales
    for launch in site["launches"]:
        folium.Marker(
            location=[site["lat"], site["lon"]],
            popup=folium.Popup(f"{site['name']}<br>Date: {launch['date']}<br>Status: {launch['status'].title()}"),
            icon=folium.Icon(color=get_icon_color(launch["status"]), icon="rocket", prefix="fa")
        ).add_to(marker_cluster)

# Agregar MousePosition para mostrar latitud/longitud
MousePosition(
    position="topright",
    separator=" | ",
    prefix="Coordenadas:",
    num_digits=5,
    empty_string="Sin posición"
).add_to(mapa)

# Guardar el mapa en un archivo HTML (opcional)
mapa.save("launches_success_failure.html")

# Mostrar el mapa en el entorno interactivo
mapa


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [12]:

import pandas as pd

# Supongamos que spacex_df ya existe, aquí tienes un ejemplo de datos
data = {
    'Flight': [1, 2, 3, 4, 5],
    'class': [1, 0, 1, 0, 1]
}

# Crear el DataFrame de ejemplo
spacex_df = pd.DataFrame(data)

# Función para asignar el color basado en el valor de 'class'
def assign_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Crear la columna 'marker_color' usando .apply()
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Mostrar el DataFrame actualizado
print(spacex_df)


   Flight  class marker_color
0       1      1        green
1       2      0          red
2       3      1        green
3       4      0          red
4       5      1        green


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [14]:
# TASK 3: Calculate the distances between a launch site to its proximities
import pandas as pd
import math

# Datos de ejemplo con coordenadas de lanzamiento y puntos de proximidad
launch_sites = {
    "site_name": ["Cape Canaveral LC-40"],
    "lat": [28.5623],
    "lon": [-80.5774]
}

proximities = {
    "name": ["Nearby Airport", "Nearby Highway", "Nearby Port"],
    "lat": [28.4294, 28.5634, 28.4100],
    "lon": [-81.3089, -80.5708, -80.6270]
}

# Crear DataFrames
launch_df = pd.DataFrame(launch_sites)
prox_df = pd.DataFrame(proximities)

# Función para calcular la distancia usando la fórmula Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Calcular distancias entre el sitio de lanzamiento y los puntos de proximidad
distances = []
for _, prox in prox_df.iterrows():
    distance = haversine(launch_df['lat'][0], launch_df['lon'][0], prox['lat'], prox['lon'])
    distances.append(distance)

# Añadir las distancias al DataFrame de proximidades
prox_df['distance_km'] = distances

# Mostrar el DataFrame resultante
print(prox_df)


             name      lat      lon  distance_km
0  Nearby Airport  28.4294 -81.3089    72.996355
1  Nearby Highway  28.5634 -80.5708     0.656070
2     Nearby Port  28.4100 -80.6270    17.615121


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
import math

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordenadas aproximadas de la costa más cercana
coastline_lat = 28.56367
coastline_lon = -80.57163

# Función para calcular la distancia usando la fórmula Haversine
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Calcular la distancia a la costa más cercana
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Mostrar el resultado
print(f"Coordenadas de la costa más cercana: Lat: {coastline_lat}, Lon: {coastline_lon}")
print(f"Distancia a la costa más cercana: {distance_coastline:.2f} km")


Coordenadas de la costa más cercana: Lat: 28.56367, Lon: -80.57163
Distancia a la costa más cercana: 0.58 km


In [18]:
import folium

# Crear un mapa centrado en el sitio de lanzamiento
mapa = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Agregar marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket")
).add_to(mapa)

# Agregar marcador para la costa más cercana
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="flag")
).add_to(mapa)

# Guardar el mapa
mapa.save("launch_site_and_coastline.html")
mapa


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
import folium

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordenadas aproximadas de la costa más cercana
coastline_lat = 28.56367
coastline_lon = -80.57163

# Crear un mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=14)

# Añadir un marcador para el sitio de lanzamiento
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket")
).add_to(site_map)

# Añadir un marcador para la costa más cercana
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="flag")
).add_to(site_map)

# Crear una línea entre el sitio de lanzamiento y la costa más cercana
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]
lines = folium.PolyLine(locations=coordinates, weight=2, color='red', dash_array='5,5')
site_map.add_child(lines)

# Guardar el mapa en un archivo HTML (opcional)
site_map.save("launch_site_to_coastline.html")

# Mostrar el mapa (si estás en un entorno interactivo como Jupyter Notebook)
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [20]:
import folium
from folium.plugins import MousePosition
from math import radians, sin, cos, sqrt, atan2

# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Datos de los puntos cercanos
nearby_points = [
    {"name": "Closest City", "lat": 28.6123, "lon": -80.8074},
    {"name": "Closest Railway", "lat": 28.5720, "lon": -80.6200},
    {"name": "Closest Highway", "lat": 28.5654, "lon": -80.5708},
    {"name": "Closest Coastline", "lat": 28.56367, "lon": -80.57163}
]

# Función para calcular la distancia usando Haversine
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Crear un mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Agregar marcador para el sitio de lanzamiento
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

# Iterar sobre los puntos cercanos para crear marcadores y líneas
for point in nearby_points:
    # Calcular la distancia
    distance = calculate_distance(launch_site_lat, launch_site_lon, point["lat"], point["lon"])

    # Crear un marcador para el punto cercano
    folium.Marker(
        location=[point["lat"], point["lon"]],
        popup=f"{point['name']}<br>Distance: {distance:.2f} km",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(site_map)

    # Dibujar una línea entre el sitio de lanzamiento y el punto cercano
    coordinates = [[launch_site_lat, launch_site_lon], [point["lat"], point["lon"]]]
    folium.PolyLine(
        locations=coordinates,
        weight=2,
        color="red",
        popup=f"Distance: {distance:.2f} km"
    ).add_to(site_map)

# Agregar MousePosition para mostrar coordenadas en tiempo real
MousePosition(
    position="topright",
    separator=" | ",
    prefix="Lat/Lon:",
    num_digits=5,
    empty_string="No position"
).add_to(site_map)

# Guardar el mapa en un archivo HTML
site_map.save("launch_site_with_nearby_points.html")

# Mostrar el mapa interactivo (Jupyter Notebook o entorno interactivo)
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
